# Simulation Workflow

In [1]:
# hide cell
%cd ..

d:\scarccpy


In [2]:
import pandas as pd
import os
import cometspy as c

from scarcc.preparation.find_directory import find_directory
from scarcc.preparation.metabolic_model import BasicModel

data_directory = find_directory('Data', os.path.abspath(''))
model_directory = find_directory('models', os.path.abspath(''))
sim_chamber_directory = find_directory('SimChamber', os.path.abspath(''))

E0, S0, all_components = BasicModel(flux_weighting=True, model_directory=model_directory).load_ES_models()
alpha_table = pd.read_csv(os.path.join(data_directory, 'alpha_table_m1.csv'), index_col=0)

p = c.params()
p.set_param("defaultKm", 0.00001) # M
p.set_param("defaultVmax", 10) #mmol/gDw/hr
p.set_param("maxCycles", 30)
# p.set_param("maxCycles", 150)
p.set_param("timeStep", 1)
p.set_param('writeFluxLog', True)


make directory at script level: d:\scarccpy\SimChamber


After configuring basic parameters, we can start to simulate the combined antibiotics. The following code will simulate the combined antibiotics for all the possible combinations of the folA and folP inhibitors. The simulation results will be saved in the directory specified by the variable sim_chamber_directory.

## Multiprocessing to derive results 
run_sim_workflow is a function that accept list of gene combinations and map each to perform simulation. Then takes the simulation output data to generate the classification of gene combination effect, concatenating all results and save to excel spreadsheet.

``df_containers`` stored all the data frames and break down of it is summarized in the last section

This example provides a minimum arguments being passed to the function
- method_list = ['m1] 
    - as suffix to the alpha_table and all the output data frames
- supply either ``SG_list`` or ``DG_list`` or both
- ``simulation_kwargs`` are passed to the class ``SimulateCombinedAntibiotics`` directly, will be discussed in the section "Breakdown of steps"

<div class="alert alert-info">

**Prerequisite:** 
    alpha_table correspond to chosen method list stored in data_directory, or run [get_alpha_biomass_df](fba_alpha_derivation.ipynb) before calling ``run_sim_workflow``
</div>

In [ ]:
from scarcc.sim_engine.simulation_workflow import run_sim_workflow

SG_list = ['folA', 'folP'] # Optional, Example: ['folA', 'folP', 'folC']
DG_list = [['folA', 'folP']] # Optional, Example: [['folA', 'folP'], ['folC', 'folP'], ['folA', 'folC']]
method_list = ['m1']
simulation_kwargs = {
    'E0': E0,
    'S0': S0,
    'base': sim_chamber_directory,
    'p': p}

df_container = run_sim_workflow(method_list=method_list, data_directory=data_directory,
                                DG_list=DG_list, **simulation_kwargs)

reading results from existing files
biomass, growth_rate, normalized_growth_rate, drug_response_classification, flux data derived from m1 were saved in d:\scarccpy\Data


In [7]:
from scarcc.sim_engine.simulation_workflow import run_sim_workflow

SG_list = ['folA', 'folP'] # Optional, Example: ['folA', 'folP', 'folC']
DG_list = [['folA', 'folP']] # Optional, Example: [['folA', 'folP'], ['folC', 'folP'], ['folA', 'folC']]
method_list = ['m1']
simulation_kwargs = {
    'E0': E0,
    'S0': S0,
    'base': sim_chamber_directory,
    'p': p}

df_container = run_sim_workflow(method_list=method_list, data_directory=data_directory,
                                DG_list=DG_list, **simulation_kwargs)

reading results from existing files
biomass, growth_rate, normalized_growth_rate, drug_response_classification, flux data derived from m1 were saved in d:\scarccpy\Data\flux_analysis_m1.csv


Customization of simulation parameters
- ``custom_simulation_kwargs`` are passed to the class ``SimulateCombinedAntibiotics``

<div class="alert alert-info">

**Note :** 
For a same set of single gene target that have run before, if "BM_SG_m1.csv" and "flux_SG_m1.csv" already exist in Data directory, and want to test new set of gene combinations, user can omit the ``SG_list`` argument. The program will use the available data that already exist.
</div>


In [5]:
from importlib import reload
import scarcc.sim_engine.simulation_workflow
reload(scarcc.sim_engine.simulation_workflow)


<module 'scarcc.sim_engine.simulation_workflow' from 'D:\\scarccpy\\src\\scarcc\\sim_engine\\simulation_workflow.py'>

In [6]:
from scarcc.sim_engine.simulation_workflow import run_sim_workflow
custom_simulation_kwargs = {
    'E0': E0,
    'S0': S0,
    'base': sim_chamber_directory, # directory to run each simulation in multiprocess
    'p': p,
    'checker_suffix': None, # checkerboard suffix for excel spreadsheet
    'ko': False, # knockout reactions if reaction is encoded by target gene

    # layout configuration parameters
    'co': True, # Run coculture simulation
    'mono': True, # Run monoculture simulation
    'mono_E': True, # Run E0 monoculture  simulation
    'mono_S': True, # Run S0 monoculture simulation
    'Smono_carbon_source': 'bulk_ac_e', # Carbon source for S0 monoculture
    'carbon_source': None, # Dictionary for specification of carbon source for each culture
    # comets model specifications
    'initial_pop': 1e-8, # Initial population for simulation
    'obj_style': 'MAX_OBJECTIVE_MIN_TOTAL', # pfba optimization
    'SG_list': ['folA', 'folP'], # List of gene targets to simulate
}

df_container = run_sim_workflow(method_list=['m1'], data_directory=data_directory,
                                DG_list=[['folA', 'folP']], **custom_simulation_kwargs)

biomass, growth_rate, normalized_growth_rate, drug_response_classification, flux data derived from m1 were saved in d:\scarccpy\Data\flux_analysis_m1.csv


## Detailed break down of steps
pseudo code for simulation procedure
```
with E0, S0:
    alter_Sij(...)
    set_comets_models(...)
    set_layout_object(...)
    sim_culture(...)  
```

<div class="alert alert-info">

**Note:** To prepare genes to perform simulation, see [Prepare Genes](prepare_single_gene_target_and_double_gene_combinations.ipynb) and [Prepare Alpha](fba_alpha_derivation.ipynb)

</div>


<div class="alert alert-warning">

Warning

Modification of stoichiometry should be within context manager (``with E0, S0:``) such that in each run, the metabolic model restores to unperturbed state

</div>

### Step 1: Stiochiometric Scaling

Reduction of biomass yield under perturbation (see [Stoichiometry Scaling](stoichiometry_scaling.ipynb))


<div class="alert alert-info">

**Note :** 
If both target gene in gene combination leads to the inhibition of the same reaction, the inhibition effect to that particular reaction is set to the maximum one rather than scaled twice

</div>

In [3]:
# Outside context manager ONLY for illustration of workflow, use with E0，S0: during implementation
from scarcc.preparation.perturbation import alter_Sij
current_gene = 'folA'
alter_Sij(E0, alphas=alpha_table.loc[current_gene, 'E0'], genes=current_gene, ko=False)
alter_Sij(S0, alphas=alpha_table.loc[current_gene, 'S0'], genes=current_gene, ko=False)

### Step 2: Construction layout object for cultures with perturbed metabolic models

In [20]:
from scarcc.sim_engine.simulation_configuration import LayoutConfig
# E0, S0 are perturbed
base_nutrient = [
    "ca2_e", "cl_e", "cobalt2_e", "cu2_e","fe2_e", "fe3_e", "k_e", "mg2_e",
    "mn2_e", "mobd_e", "ni2_e", "o2_e", "pi_e", "so4_e", "zn2_e", "nh4_e"]

layout_kwargs = {
    'E0': E0, 'S0': S0, 'carbon_source_val': .1, 'base_nutrients': base_nutrient, 
    'nutrients_val': 100, 'Smono_carbon_source': 'bulk_ac_e'
}
layout_config = LayoutConfig(**layout_kwargs)
layout_config.set_comets_model()
co_layout, E0_layout, S0_layout = layout_config.set_layout_object()

Or pass different carbon sources inside a dictionary for each cultures with 'co', 'mono_E', 'mono_S' as keys

In [25]:
layout_kwargs['carbon_source'] = carbon_source = {'co': 'lcts_e', 'mono_E': 'lcts_e', 'mono_S': 'bulk_ac_e'}
layout_config = LayoutConfig(**layout_kwargs)
layout_config.set_comets_model()
co_layout, E0_layout, S0_layout = layout_config.set_layout_object()

### Step 3: Run simulation with layout

In [32]:
sim = c.comets(co_layout, p)
sim.run()


Running COMETS simulation ...
Done!


Extract biomass and flux data frames from COMETS sim object

In [40]:
print('biomass: \n', sim.total_biomass.head(), 
      '\n E0 fluxes:\n', sim.fluxes_by_species['E0'].head(),
      '\n S0 fluxes:\n', sim.fluxes_by_species['S0'].head())

biomass: 
    cycle            E0            S0
0      0  1.000000e-08  1.000000e-08
1      1  1.000000e-08  1.000000e-08
2      2  1.000000e-08  1.000000e-08
3      3  1.000000e-08  1.014568e-08
4      4  1.047327e-08  1.014568e-08 
 E0 fluxes:
    cycle  x  y  CYTDK2  XPPT  ...  MOCOS  BMOGDS2  FESD2s  OCTNLL  DHFR_v1
0      5  1  1     0.0   0.0  ...    0.0      0.0     0.0     0.0      0.0
2     10  1  1     0.0   0.0  ...    0.0      0.0     0.0     0.0      0.0
4     15  1  1     0.0   0.0  ...    0.0      0.0     0.0     0.0      0.0
6     20  1  1     0.0   0.0  ...    0.0      0.0     0.0     0.0      0.0
8     25  1  1     0.0   0.0  ...    0.0      0.0     0.0     0.0      0.0

[5 rows x 2716 columns] 
 S0 fluxes:
    cycle  x  y   3HAD180  ...  OA4L_ST       OA5L_ST  OA4VL_ST  DHFR_v1
1      5  1  1  0.000092  ...      0.0  1.990201e-05       0.0      0.0
3     10  1  1 -0.008731  ...      0.0  5.364746e-07       0.0      0.0
5     15  1  1 -0.000136  ...      0.0  6.088823

## Integration of all steps

### Streamlined Process for Simulation Workflow 
The implementations illustrated above are integrated in the class ``SimulateCombinedAntibiotics``
- takes a target gene or a tuple of gene combination as input
- performs a simulation, and returns the biomass and flux results

In [3]:
from scarcc.sim_engine.simulation_workflow import SimulateCombinedAntibiotics
alpha_table = pd.read_csv(os.path.join(data_directory, 'alpha_table_m1.csv'), index_col=0)
cas = SimulateCombinedAntibiotics(current_gene=['folP','folA'], E0=E0, S0=S0, alpha_table=alpha_table,
                                p=p, mono=True, base=sim_chamber_directory)

biomass_df, flux_df = cas.get_BM_df()


Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!


## Display of data frames inside ``df_container``

All the result data frames can be queried with tuple key: ``(method_chosen, XG)``

where method chosen is any method in the method_list and XG is either 'SG' or 'DG'

In [5]:
df_container[('m1', 'DG')].keys()

dict_keys(['biomass', 'flux', 'growth_rate', 'normalized_growth_rate', 'drug_response_classification'])

- "growth_rate" and "drug_response_classification" are generated in class ``MethodDataFiller``
    - To import, run: 
    
        ``from scarcc.sim_engine.output import MethodDataFiller``
    - Initialize a df_container and pass to MethodDataFiller:
    
        pass a df_container with keys "(method_chosen, XG)"
        for each XG, provide a dictionary with keys "biomass" and "flux" and the values as the corresponding data frame

Example:

```
df_container = {}
df_container['m1', 'SG']  = {'biomass': SG_biomass_df,
                            'flux': SG_flux_df}
df_container['m1', 'DG']  = {'biomass': DG_biomass_df,
                            'flux': DG_flux_df}
MethodDataFiller(df_container)
mdf.fill_container()
mdf.write_to_csv()

```

SG data frames

In [30]:
pd.set_option('display.max_columns', 3)
[print(k, ': \n', df.head(), '\n') for k, df in df_container[('m1', 'SG')].items()]

biomass : 
        E0_Normal_coculture  S0_Normal_coculture  E0_Normal_monoculture  \
cycle                                                                    
0             1.000000e-08         1.000000e-08           1.000000e-08   
1             1.000000e-08         1.000000e-08           1.754153e-08   
2             1.000000e-08         1.000000e-08           3.077053e-08   
3             1.000000e-08         1.029133e-08           5.397622e-08   
4             1.094643e-08         1.029133e-08           9.468255e-08   

       S0_Normal_monoculture  E0_folA_coculture  S0_folA_coculture  \
cycle                                                                
0               1.000000e-08       1.000000e-08       1.000000e-08   
1               1.077695e-08       1.000000e-08       1.000000e-08   
2               1.161426e-08       1.000000e-08       1.000000e-08   
3               1.251663e-08       1.000000e-08       1.014568e-08   
4               1.348911e-08       1.047327e-08  

[None, None, None, None]

DG data frames

In [8]:
[print(k, ': \n', df.head(), '\n') for k, df in df_container[('m1', 'DG')].items()]

biomass : 
        E0_folA.folP_coculture  S0_folA.folP_coculture  \
cycle                                                   
0                1.000000e-08            1.000000e-08   
1                1.000000e-08            1.000000e-08   
2                1.000000e-08            1.000000e-08   
3                1.000000e-08            1.000954e-08   
4                1.000217e-08            1.000954e-08   

       E0_folA.folP_monoculture  S0_folA.folP_monoculture  
cycle                                                      
0                  1.000000e-08              1.000000e-08  
1                  1.000217e-08              1.003103e-08  
2                  1.000434e-08              1.006215e-08  
3                  1.000650e-08              1.009337e-08  
4                  1.000867e-08              1.012469e-08   

flux : 
                                      cycle  x  y  CYTDK2  XPPT  HXPRT  NDPK5  \
Gene_inhibition Species culture                                              

[None, None, None, None, None]